# Sample QOCT calculations with the Krotov algorithm

In [ ]:
try:
    get_ipython
    isnotebook = True
except:
    isnotebook = False

import os
import sys
import numpy as np
import math
import scipy as sp
import matplotlib
if not isnotebook:
    matplotlib.use('Agg')
import matplotlib.pyplot as plt
import time
import nlopt
from qutip import *

In [ ]:
import qocttools
import qocttools.models.GdW30 as GdW30
import qocttools.hamiltonians as hamiltonians
import qocttools.math_extra as math_extra
import qocttools.pulses as pulses
import qocttools.qoct as qoct
import qocttools.solvers as solvers
import qocttools.target as target

In [ ]:
qocttools.about()

In [ ]:
data = []

# Introduction

In this notebook we show some simple examples of optimization done with QOCT, using the math_extra.maximize routine.

# Model

The model is defined by the Hamiltonian:

\begin{equation}
        \hat{H}(t) = \hat{H}_0 + f(t)\hat{V}
\end{equation}
where the time-independent part is given by:
\begin{equation}
        \hat{H}_0 = D\bigg[\hat{S}_z^2 - \frac{1}{3}S(S + 1)\bigg] + E[\hat{S}_x^2 - \hat{S}_y^2] - g\mu_B\hat{\vec{S}}\cdot\vec{H}
\end{equation}
and the time-dependent part is:
\begin{equation}
        \hat{H}(t) = \hat{H}_0 + f(t)\hat{V}
\end{equation}
The perturbation is a magnetic field:
\begin{equation}
        \hat{V} = -g\mu_B\hat{\vec{S}}\cdot\vec{H}_m 
\end{equation}

In this case:

* $S = 7/2$

* $D$ = 1281 MHz

* $E$ = 294 MHz

* $\vec{H} = (0.15, 0.0, 0.0)$ T

* $\vec{H}_m = (0, 0.001, 0.0)$ T

In [ ]:
S = 7/2 # spin
E = 294 # value in MHz
D = 1281 # value in MHz
dim = int(2*S + 1) #matrix dim

In [ ]:
H = np.array([0.15, 0, 0.0], dtype = float) #magnetic field in T
H_m = np.array([0, 0.001, 0], dtype = float) #only in presence of perturbation (T)
H0 = GdW30.hGdW30(D, E, H)
V = GdW30.vGdW30(H_m)

In [ ]:
eigenvalues, eigenstates = H0.eigenstates()
H0 = H0.transform(eigenstates) - eigenvalues[0]

In [ ]:
# In principle, we could just transform V with the recently obtained eigenstates. Unfortunately, that
# would make the test results different in different computers, as the eigenstates can have different phases.
# Therefore, we use the V that was computed once, and stored in file "V". The commented code was used
# to generate and store V.
#V = V.transform(eigenstates)
#fileio.file_data_store("V", V, numformat = 'exp')
V = Qobj(fileio.file_data_read("V"))

In [ ]:
H = hamiltonians.hamiltonian(H0, [V])

In [ ]:
w = np.zeros(dim-1)
taui = np.zeros(dim-1)
for i in range(dim-1):
    w[i] = eigenvalues[i+1] - eigenvalues[i]
    taui[i] = 2.0*np.pi/w[i]
    print("Transition {:d}: w = {:f} MHz, tau = {:f} ns".format(i, w[i], 1000.0*taui[i]/(2.0*np.pi)))

# Time array definition

In [ ]:
T = 10 * taui[0]
print("T = {:f} us*2*pi = {:f} ns".format(T, 1000*T/(2.0*np.pi)))
time = math_extra.timegrid(H0, T, 1.0)
print('#Time steps =', time.shape[0])

# Control function

The control function is parametrized with the Fourier expansion as follow:
\begin{equation}
    f(u, t) = \frac{1}{\sqrt{T}}u_0 + \frac{2}{\sqrt{T}}\sum_{k = 1}^{M}u_{2k}\cos(\omega_kt) + \frac{2}{\sqrt{T}}\sum_{k = 1}^{M}u_{2k + 1}\sin(\omega_kt),
\end{equation}
where $u_0\dots u_{2M + 1}$ are the control parameters. This way, we can compute the derivate respect any control parameter as
\begin{equation}
    \frac{\partial f}{\partial u_m}(u, t) = f(e_m, t),
\end{equation}
where $e_m$ is the set of parameters where all of them are zero except the m-th ane, that is equal to one.

This pulse parametrization is included in the typical_pulses.py file as pulse class.

In [ ]:
# First, we will build a pulse as a Fourier series.

M = 10
omega = np.zeros(M+1)
omega[0] = 0.0
for k in range(1, M+1):
    omega[k] = (2.0*np.pi/T) * k
    #print("omega[{:d}] = {:f} MHz".format(k, omega[k]))

#u = 1.0*np.random.rand((2*M + 1))
u = np.zeros(2*M+1)
u[2] = 1.0
f = pulses.pulse("fourier", T, u = u)

# Then, we transform this into the real time parametrization.
ft = pulses.pulse("realtime", T, u = f.fu(time))
u = f.fu(time)

In [ ]:
fig, ax = plt.subplots()

ax.plot(time * 1000/(2.0*np.pi), ft.fu(time))
ax.set_xlabel("Time (ns)")
ax.set_xlim(left = 0.0, right = time[-1]*1000/(2.0*np.pi))
ax.set_ylabel("f(t) (mT)")
if isnotebook:
    plt.show()
else:
    fig.savefig("pulse.pdf")

# State optimization 

The target will be the population of the first excited state.
\begin{equation}
T(\psi(T)) = \vert \langle 1 \vert \psi(T)\rangle\vert^2
\end{equation}
We will use a penalty function:
\begin{equation}
P(u) = -\alpha \int_0^T\!\!{\rm d}t\; f^2(u, t)\,.
\end{equation}

Therefore, the target functionl is:
\begin{equation}
F(\psi(T), u) = T(\psi(T)) + P(u)\,.
\end{equation}

## QOCT target function definition

In [ ]:
def S(t):
    r = 10.0
    return sp.special.erf(r*t/T) * sp.special.erf(r*(T-t)/T)

In [ ]:
fig, ax = plt.subplots()
ax.plot(time, S(time))
plt.show()

In [ ]:
target_level = 1
alpha = 1.0

def Pfunction(u):
    f = pulses.pulse("realtime", T, u = u)
    return - alpha * sp.integrate.simps(f.fu(time[1:-1]) * f.fu(time[1:-1]) / S(time[1:-1]), time[1:-1])

state_0 = basis(dim, 0)

In [ ]:
tg = target.Target('expectationvalue',
                   operator = fock_dm(dim, target_level), S = S, alpha = alpha)

In [ ]:
opt = qoct.Qoct(H, T, time.shape[0], tg, ft, state_0,
                interaction_picture = True,
                solve_method = 'rk4')

In [ ]:
x, optval, res = opt.maximize(algorithm = -1, maxeval = 10, verbose = True)
data.append(optval)

In [ ]:
ft.set_parameters(x)
G = opt.gfunc(x)
P = Pfunction(x)
print(G - P, P, G)

In [ ]:
fig, ax = plt.subplots()

ft.set_parameters(u)
ax.plot(time * 1000/(2.0*np.pi), ft.fu(time), label = 'initial guess')
ft.set_parameters(x)
ax.plot(time * 1000/(2.0*np.pi), ft.fu(time), label = 'optimal pulse')
ax.set_xlabel("Time (ns)")
ax.set_xlim(left = 0.0, right = time[-1]*1000/(2.0*np.pi))
ax.set_ylabel("f(t) (mT)")
ax.legend()

if isnotebook:
    plt.show()
else:
    fig.savefig("pulse.pdf")

# Gate optimization

Now, the target will be the creation of the Toffoli quantum gate. We will not use a penalty function, but set bounds for the coefficients values.

In [ ]:
ft = pulses.pulse("realtime", T, u = f.fu(time))
u = ft.fu(time)

## QOCT target function definition

In [ ]:
U_target = qip.operations.toffoli()
U_0 = qeye(dim) #initial state

In [ ]:
tg = target.Target('evolutionoperator', operator = U_target, Utarget = U_target, 
                   S = S, alpha = 0.1 * alpha)

opt = qoct.Qoct(H, T, time.shape[0], tg, ft, U_0,
                interaction_picture = True,
                solve_method = 'rk4')

In [ ]:
x, optval, res = opt.maximize(algorithm = -1, maxeval = 10, verbose = True)
data.append(optval)

In [ ]:
with open("data", "w") as datafile:
     for i in data:
         datafile.write("{:.14e}\n".format(i))